In [1]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer

In [2]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
PRE_TRAINED_MODEL_NAME = 'bert-base-cased'

In [3]:
imdb_data=pd.read_csv('./data/processed_train.csv')
print(imdb_data.shape)
imdb_data.head(5)

(29341, 4)


,ID,review,sentiment,processed_review
0,41411,I watched this film because I'm a big fan of R...,0,watch film m big fan river phoenix joaquin pho...
1,37586,It does not seem that this movie managed to pl...,1,movie manage lot people see place bump acciden...
2,6017,"Enough is not a bad movie , just mediocre .",0,bad movie mediocre
3,44656,my friend and i rented this one a few nights a...,0,friend rent night ago single good movie see me...
4,38711,"Just about everything in this movie is wrong, ...",0,movie wrong wrong wrong mike myers example s r...


In [4]:
imdb_test_data=pd.read_csv('./data/processed_test.csv')
print(imdb_test_data.shape)
imdb_test_data.head(5)

(29341, 3)


,ID,review,processed_review
0,22622,Robert Lansing plays a scientist experimenting...,robert lansing play scientist experiment pass ...
1,10162,"Well I've enjoy this movie, even though someti...",ve enjoy movie turn stereotypical situation nt...
2,17468,First things first - though I believe Joel Sch...,thing believe joel schumacher well mediocre di...
3,42579,I watched this movie on the grounds that Amber...,watch movie ground amber benson rock nick stah...
4,701,A certain sexiness underlines even the dullest...,certain sexiness underline dull tangent


In [23]:
%%time
# Concatenate train data and test data
all_review = np.concatenate([imdb_data.processed_review.values, imdb_test_data.processed_review.values])

CPU times: user 0 ns, sys: 1.62 ms, total: 1.62 ms
Wall time: 1.57 ms


### Choose MAX_LENGTH for BERT

In [ ]:
token_lens = []

for txt in all_review:
    tokens = tokenizer.encode(txt, max_length=512)
    token_lens.append(len(tokens))
    

In [ ]:
sns.distplot(token_lens)
plt.xlim([0, 256]);
plt.xlabel('Token count')

In [ ]:
MAX_LEN = 

In [22]:
encoded_sent = tokenizer.encode_plus(
    text=imdb_data.loc[7,'processed_review'],  # Preprocess sentence
    add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
    max_length=MAX_LEN,                  # Max length to truncate/pad
    pad_to_max_length=True,         # Pad sentence to max length
    return_tensors='pt',           # Return PyTorch tensor
    return_attention_mask=True      # Return attention mask
    )
encoded_sent.keys()

NameError: name 'MAX_LEN' is not defined

In [5]:
# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=data,  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [10]:
len(all_review)

58682

In [11]:
all_review[15]

'inept big screen remake avenger wild wild west'

In [16]:
# tokenize sample
tokens = tokenizer.tokenize(all_review[18])
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print(tokens)
print(tokens_ids)

['answer', 'person', 's', 'question', 'doesn', 'remember', 'film', 's', 'people', 'see', '1978', 's', 'show', 'tv', 's', 'video', 'd', 'news', 'era', 'mind', '##less', 'comedy', '70', 'movie', 'go', '##er', 'smart', 'avoid', 'film', 'love', 'billy', 'cry', '##tal', 'paul', 'l', '##yn', '##de', 'joan', 'river', 'paul', 'l', '##yn', '##de', 'funny', 'be', '##wi', '##tch', 'hollywood', 'square', 'joan', 'river', 'time', 'career', 'get', 'laugh', 'make', 'cruel', 'joke', 'singer', 'karen', 'carpenter', 's', 'lack', 'weight', 'ha', '##r', 'ha', '##r', 'joan', 'like', 'somewhat', 'famous', 'era', 'cast', 'surprising', 'doris', 'robert', 'later', 'everybody', 'love', 'raymond', 'somewhat', 'good', 'idea', 'storyline', 'man', 'get', 'pregnant', 'instead', 'woman', 'go', 'waste', 'help', 'male', 'friend', 'crystal', 'get', 'set', 'hooker', 'finally', 'lose', 'virginity', 'instead', 'get', 'pregnant', 'commentary', 'woman', 'take', 'position', 'power', 'away', 'man', 'crystal', 's', 'stomach', '

In [7]:
%%time

# Encode our concatenated data
encoded_review = [tokenizer.encode(sent, add_special_tokens=True) for sent in all_review]

# Find the maximum length
max_len = max([len(sent) for sent in encoded_review])
print('Max length: ', max_len)

ValueError: Input nan is not valid. Should be a string, a list/tuple of strings or a list/tuple of integers.

In [ ]:
# Specify `MAX_LEN`
MAX_LEN = 64

# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([X[0]])[0].squeeze().numpy())
print('Original: ', X[0])
print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)